In [1]:
from path import path
import os
rootdir = path(u'/home/pfernique/Desktop/PyClangLite')
for filepath in (rootdir/'src'/'wrapper').walkfiles('*.[hc]*'):
    os.remove(filepath.abspath())

In [2]:
import subprocess
popen = subprocess.Popen(['clang++', '-x', 'c++', '-v', '-E', '/dev/null'], stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE)
out, err = popen.communicate()
sysincludes = err.splitlines()
headers = []
flags = ['-x', 'c++', '-std=c++11', '-D__STDC_LIMIT_MACROS', '-D__STDC_CONSTANT_MACROS']
if '#include <...> search starts here:' not in sysincludes or 'End of search list.' not in sysincludes:
    warnings.warn('System includes not computed: parsing clang command output failed', Warning)
else:
    lindex = sysincludes.index('#include <...> search starts here:')
    rindex = sysincludes.index('End of search list.')
    sysincludes = sysincludes[lindex+1:rindex]
flags.extend(['-I'+str(path(sysinclude.strip()).abspath()) for sysinclude in sysincludes])
for sysinclude in sysincludes:
    sysinclude = path(sysinclude.strip())
    clanginclude = sysinclude/'clang'
    if clanginclude.exists():
        headers.extend([sysinclude.relpathto(header) for header in clanginclude.walkfiles('*.h')])
selfcontained = rootdir/'src'/'wrapper'/'clang.h'
selfcontained.write_lines(['#include <' + str(header) + '>' for header in headers])
headers  = [selfcontained]

In [3]:
from autowig import autowig
import pickle
try:
    with open('pyclanglite.pkl', 'r') as f:
        asg = pickle.load(f)
except:
    asg = autowig.AbstractSemanticGraph()
    autowig.parser.plugin = 'pyclanglite'
    autowig.parser(asg, headers, flags=flags, bootstrap=False)
    with open('pyclanglite.pkl', 'w') as f:
        pickle.dump(asg, f)

In [4]:
controller = """\
def bootstrap_controller(asg):
    node = asg['class ::std::unique_ptr']
    node.is_copyable = False
    node.is_smart_pointer = True
"""
exec controller + "    return asg"
autowig.controller['bootstrap'] = bootstrap_controller
autowig.controller.plugin = 'bootstrap'
asg = autowig.controller(asg)

In [5]:
subset = asg.nodes('::clang::tooling::buildASTFromCodeWithArgs')
classes = [asg['class ::clang::Type'], asg['class ::clang::Decl']]
subset += classes
subset += classes[0].subclasses(recursive=True)
subset += classes[1].subclasses(recursive=True)

In [6]:
#-Wl,--unresolved-symbols=report-all
from autowig.boost_python_generator import caching
caching(asg)

In [7]:
autowig.generator.plugin = 'boost_python'
wrappers = autowig.generator(asg, subset,
                             module=rootdir/'src'/'wrapper'/'_pyclanglite.cpp',
                             decorator=None, closure=True)
if autowig.parser == 'libclang':
    pass

In [ ]:
for wrapper in wrappers:
    wrapper.write()
session = autowig.scons(rootdir, '-k', '-j5')

In [20]:
print autowig.feedback(session, rootdir, asg,
                            variantdir='build-scons',
                            tabsize=4)

for method in asg['class ::llvm::ArrayRef<clang::CharSourceRange>'].methods(access='public'):
    if method.prototype == 'bool  equals(class ::llvm::ArrayRef<clang::CharSourceRange> ) const':
        method.boost_python_export = False
        break
for constructor in asg['class ::clang::FileScopeAsmDecl'].constructors(access='public'):
    if constructor.prototype == 'FileScopeAsmDecl(class ::clang::FileScopeAsmDecl  const &)':
        constructor.boost_python_export = False
        break
asg['class ::clang::FileScopeAsmDecl'].is_copyable = False
for method in asg['class ::llvm::ArrayRef<clang::FixItHint>'].methods(access='public'):
    if method.prototype == 'bool  equals(class ::llvm::ArrayRef<clang::FixItHint> ) const':
        method.boost_python_export = False
        break
for constructor in asg['class ::clang::Stmt'].constructors(access='public'):
    if constructor.prototype == 'Stmt(enum ::clang::Stmt::StmtClass )':
        constructor.boost_python_export = False
        break

In [10]:
controller += autowig.feedback(session, rootdir, asg,
                            variantdir='build-scons',
                            tabsize=4, indent=1)
exec controller + "    return asg"
autowig.controller['bootstrap'] = bootstrap_controller
autowig.controller.plugin = 'bootstrap'

IndentationError: expected an indented block (<string>, line 7)

In [16]:
controller

"def bootstrap_controller(asg):\n    node = asg['class ::std::unique_ptr']\n    node.is_copyable = False\n    node.is_smart_pointer = True\n"

In [ ]:
err = """scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
g++ -o build-scons/src/wrapper/export_llvm_iterator_adaptor_base_0de29d52f7ba54808f673176f20563c2.os -c -DNDEBUG -O2 -std=c++0x -fpermissive -DNDEBUG -O2 -fPIC -ftemplate-depth-100 -DBOOST_PYTHON_DYNAMIC_LIB -Ibuild-scons/include -I/usr/include -I/usr/include/python2.7 -I/usr/local/include build-scons/src/wrapper/export_llvm_iterator_adaptor_base_0de29d52f7ba54808f673176f20563c2.cpp
In file included from /usr/local/include/clang/Lex/PreprocessingRecord.h:23:0,
                 from /home/pfernique/Desktop/PyClangLite/src/wrapper/clang.h:1,
                 from build-scons/src/wrapper/export_llvm_iterator_adaptor_base_0de29d52f7ba54808f673176f20563c2.cpp:3:
/usr/local/include/llvm/ADT/iterator.h: In instantiation of 'ReferenceT llvm::iterator_adaptor_base<DerivedT, WrappedIteratorT, IteratorCategoryT, T, DifferenceTypeT, PointerT, ReferenceT, WrappedTraitsT>::operator*() const [with DerivedT = clang::DeclContext::udir_iterator; WrappedIteratorT = clang::DeclContextLookupResult::iterator; IteratorCategoryT = std::random_access_iterator_tag; T = clang::UsingDirectiveDecl*; DifferenceTypeT = long int; PointerT = clang::UsingDirectiveDecl**; ReferenceT = clang::UsingDirectiveDecl*&; WrappedTraitsT = std::iterator_traits<clang::DeclContextLookupResult::iterator>]':
build-scons/src/wrapper/export_llvm_iterator_adaptor_base_0de29d52f7ba54808f673176f20563c2.cpp:21:716:   required from here
/usr/local/include/llvm/ADT/iterator.h:216:42: error: cannot bind rvalue '(clang::UsingDirectiveDecl*)((clang::NamedDecl*)((const llvm::iterator_adaptor_base<clang::DeclContext::udir_iterator, clang::DeclContextLookupResult::iterator, std::random_access_iterator_tag, clang::UsingDirectiveDecl*>*)this)->llvm::iterator_adaptor_base<clang::DeclContext::udir_iterator, clang::DeclContextLookupResult::iterator, std::random_access_iterator_tag, clang::UsingDirectiveDecl*>::I.clang::DeclContextLookupResult::iterator::operator*())' to 'clang::UsingDirectiveDecl*&'
   ReferenceT operator*() const { return *I; }
                                          ^
scons: *** [build-scons/src/wrapper/export_llvm_iterator_adaptor_base_0de29d52f7ba54808f673176f20563c2.os] Error 1
scons: building terminated because of errors.
"""

In [ ]:
import parse
for line in err.splitlines():
    #print line
    parsed = parse.parse('build-scons/{filename}:{row}:{column}:   required from here', line)
    if parsed:
        row = int(parsed['row'])
        print row, parsed['filename']

In [ ]:
line = asg[rootdir/'src/wrapper/export_llvm_iterator_adaptor_base_0de29d52f7ba54808f673176f20563c2.cpp'].content.splitlines()[20]
print line

In [ ]:
import parse
res = parse.parse('    {return_type} ({pointer})(){cv}= {globalname};', line)

In [ ]:
res.named.get("parameters", "")